In [1]:
!pip install scikit-surprise

In [2]:
import pandas as pd
import numpy as np
import surprise
import os

In [3]:
ratings = pd.read_table("C:/Users/dbda/Desktop/PML/recommender_systems/ml-100k/u.data",sep='\t',names = ['user_id','item_id','rating','timestamp'])
ratings.drop('timestamp',axis=1,inplace=True)
ratings.head()

,user_id,item_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


In [4]:
lowest_rating = ratings['rating'].min()
highest_rating = ratings['rating'].max()
print("Ratings range between {0} and {1}".format(lowest_rating,highest_rating))

Ratings range between 1 and 5


In [5]:
reader = surprise.Reader(rating_scale = (lowest_rating,highest_rating))
data = surprise.Dataset.load_from_df(ratings,reader)
type(data)

surprise.dataset.DatasetAutoFolds

Tuning for best parameters

In [6]:
from surprise.model_selection import GridSearchCV
from surprise.model_selection.split import KFold

param_grid = {'lr_all':np.linspace(0.001,1,3), 'reg_all': np.linspace(0.01,0.8,3),
              'n_factors':[40,30]}
kfold = KFold(random_state=23,n_splits=5,shuffle=True)
gs = GridSearchCV(surprise.SVD, param_grid,joblib_verbose =3, measures=['rmse', 'mae'], cv=kfold,n_jobs=-1)
gs.fit(data)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:   10.2s finished


Best Score:

In [7]:
print(gs.best_score['rmse'])

0.9603410408355572


Best Parameter:

In [8]:
print(gs.best_params['rmse'])

{'lr_all': 0.001, 'reg_all': 0.01, 'n_factors': 30}


We can now use the algorithm that yields the best rmse:

In [9]:
algo = gs.best_estimator['rmse']
algo.fit(data.build_full_trainset())

The recommendations can be generated for any user with the object **algo**.

Total Items:

In [11]:
iids = ratings['item_id'].unique()
print(iids)

[ 242  302  377 ... 1637 1630 1641]


The list of items rated by user 50:

In [12]:
u_iid = ratings[ratings['user_id']==50]['item_id'].unique()
print(u_iid)

[ 246  823  253  475 1084  286    9  125  123  325  508  288  319  324
  276 1008 1010  268  544   15  327  124  547  100]


List of the items not rated by user 50:

In [13]:
iids_to_predict = np.setdiff1d(iids, u_iid)
print(iids_to_predict)

[   1    2    3 ... 1680 1681 1682]


Extracting the estimated rating from iids_to_predict

In [14]:
testset = [[50,iid,0.] for iid in iids_to_predict]
predictions = algo.test(testset)

In [15]:
predictions[0]

Prediction(uid=50, iid=1, r_ui=0.0, est=3.847588728522284, details={'was_impossible': False})

In [16]:
pred_ratings = np.array([pred.est for pred in predictions])

Getting the item with highest expected rating

In [17]:
i_max = np.argmax(pred_ratings)
print("Item:",iids_to_predict[i_max],"is the item with highest expected rating as",pred_ratings[i_max])

Item: 64 is the item with highest expected rating as 4.4758219524942175


In [18]:
exp_ratings = pd.DataFrame({'Item_ID': iids_to_predict, 'Exp_Rating':pred_ratings})
exp_ratings.sort_values(by=['Exp_Rating','Item_ID'], ascending=[False, True], inplace=True)
exp_ratings.head(10)

,Item_ID,Exp_Rating
61,64,4.475822
305,318,4.465052
162,169,4.447652
391,408,4.431967
10,12,4.418889
582,603,4.415678
47,50,4.396456
465,483,4.372132
496,515,4.351063
262,272,4.334147
